In [17]:
import cv2
import json
import matplotlib.pyplot as plt
import os
import numpy as np

In [4]:
def load_json(json_file):

    with open(json_file, "r") as f:
        js_data = json.load(f)

    res = []

    for object in js_data["Objects"]:
        label = object["Label"]
        one_res = {
            "class": label,
        }    
        if "Points" in object:
            one_res["type"] = "polygon"
            polygon = [[round(x), round(y)] for x,y in object["Points"]]
            one_res["polygon"] = polygon
        elif "Box" in object:
            one_res["type"] = "bbox"
            bbox = [round(x) for x in object["Box"]]
            one_res["bbox"] = ((bbox[0], bbox[1]), (bbox[2], bbox[3]))
        else:
            raise ValueError(f"object: {object}")

        res.append(one_res)

    return res


In [11]:
def draw_polygon(img, polygon, color, label=None, line_thickness=2):
    assert img.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(img) to plot_on_box() input image.'
    c1 = tuple(polygon[0])
    polygon = np.array(polygon)
    polygon = polygon[:, np.newaxis, :]
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    cv2.polylines(img, [polygon], isClosed=True, color=color, thickness=tl, lineType=cv2.LINE_AA)

    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tl)[0]
        c2 = (c1[0] + t_size[0], c1[1] - t_size[1] - 3)
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
        
def draw_bbox(img, bbox, color, label=None, line_thickness=2):
    """
	args: 
         * img: np.array
         * bbox: tuple, ((x1, y1), (x2, ye)), int
         * color: tuple, int
         * label: str
         * line_thickness: int
    """
    assert img.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(img) to plot_on_box() input image.'
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    c1, c2 = bbox[0], bbox[1]
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tl)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)



class Colors:
    # Ultralytics color palette https://ultralytics.com/
    def __init__(self):
        # hex = matplotlib.colors.TABLEAU_COLORS.values()
        hexs = ('FF3838', 'FF9D97', 'FF701F', 'FFB21D', 'CFD231', '48F90A', '92CC17', '3DDB86', '1A9334', '00D4BB',
                '2C99A8', '00C2FF', '344593', '6473FF', '0018EC', '8438FF', '520085', 'CB38FF', 'FF95C8', 'FF37C7')
        self.palette = [self.hex2rgb(f'#{c}') for c in hexs]
        self.n = len(self.palette)


    def __call__(self, i, bgr=False):
        c = self.palette[int(i) % self.n]
        return (c[2], c[1], c[0]) if bgr else c


    @staticmethod
    def hex2rgb(h):  # rgb order (PIL)
        return tuple(int(h[1 + i:1 + i + 2], 16) for i in (0, 2, 4))

colors = Colors()  # create instance for 'from utils.plots import colors'

def get_bbox(polygon):
    
    x1 = min([x for x,_ in polygon])
    y1 = min([y for _,y in polygon])
    x2 = max([x for x,_ in polygon])
    y2 = max([y for _,y in polygon])
    
    return ((x1, y1), (x2, y2))
    
    

In [3]:
dataset_root = "/media/yunfei/mm2/cicvdata/che_seg_road/seg"

file_list = []
img_root = os.path.join(dataset_root, "images")
js_root = os.path.join(dataset_root, "jsons")

for x in os.listdir(img_root):
    img_path = os.path.join(img_root, x)
    js_path = os.path.join(js_root, x.rsplit(".", 1)[0] + ".json")
    assert os.path.exists(js_path)
    
    file_list.append([img_path, js_path])

len(file_list), file_list[0]

(553,
 ['/media/yunfei/mm2/cicvdata/che_seg_road/seg/images/1668403425.jpg',
  '/media/yunfei/mm2/cicvdata/che_seg_road/seg/jsons/1668403425.json'])

In [9]:
class_names = """S0001
S0002
S0003
S0075
S0076
S0077
S0078
S0079
S0083
S0086
"""
class_names = {x:i for i,x in enumerate(class_names.split())}
data = []
for img_path, js_path in file_list:
    js_data = load_json(js_path)
    tmp = []
    for one_label in js_data: # print(js_data)
        class_name = one_label["class"]
        if class_name not in class_names:
            raise
            # class_names[class_name] = len(class_names)
        
        class_idx = class_names[class_name]
        assert one_label["type"] == "polygon", one_label
        polygon = one_label["polygon"]
        tmp.append([class_idx, polygon])
    data.append([img_path, tmp])

data[0],class_names

(['/media/yunfei/mm2/cicvdata/che_seg_road/seg/images/1668403425.jpg',
  [[0,
    [[1594, 1119],
     [1586, 1119],
     [1527, 1091],
     [1506, 1080],
     [1360, 939],
     [1116, 697],
     [1125, 695],
     [1238, 799],
     [1435, 976],
     [1586, 1110]]],
   [0, [[1121, 692], [1112, 694], [1089, 670], [1080, 660], [1086, 658]]],
   [0,
    [[0, 940],
     [0, 952],
     [126, 909],
     [369, 827],
     [618, 743],
     [792, 683],
     [787, 680],
     [675, 718],
     [574, 752],
     [384, 814],
     [226, 865]]],
   [0, [[792, 679], [801, 680], [893, 649], [891, 646], [796, 678]]],
   [1,
    [[594, 1041],
     [579, 1047],
     [546, 1048],
     [546, 1046],
     [699, 905],
     [727, 906],
     [728, 908],
     [641, 997]]],
   [1,
    [[798, 838],
     [776, 838],
     [772, 837],
     [776, 834],
     [826, 790],
     [842, 790],
     [841, 792]]],
   [1,
    [[873, 762], [858, 761], [858, 760], [882, 736], [892, 738], [893, 739]]],
   [1, [[910, 723], [897, 722], [89

In [13]:
idx2name = {b:a for a,b in class_names.items()}

In [19]:
save_root = "/home/yunfei/tmp/aaa"
os.makedirs(save_root, exist_ok=True)

for img_path, label in data:
    img = cv2.imread(img_path)
    for class_idx, polygon in label:
        color = colors(class_idx)
        draw_str = str(class_idx) + "/" + idx2name[class_idx]
        #bbox = get_bbox(polygon)
        
        draw_polygon(img, polygon, color, draw_str)
    save_path = os.path.join(save_root, os.path.basename(img_path))
    cv2.imwrite(save_path, img)

In [22]:
count = {a:0 for a in class_names}
for img_path, label in data:
    for x, _ in label:
        name = idx2name[x]
        count[name] += 1
        
count

{'S0001': 2098,
 'S0002': 3995,
 'S0003': 25,
 'S0075': 1117,
 'S0076': 86,
 'S0077': 32,
 'S0078': 42,
 'S0079': 99,
 'S0083': 39,
 'S0086': 30}

In [24]:
print("生成mask.png 标签")

save_root = os.path.join(dataset_root, "labels-mask/")
os.makedirs(save_root, exist_ok=True)

print("保存目录：", save_root)

for img_path, label in data:
    img = cv2.imread(img_path)
    img_h, img_w = img.shape[:2]
    mask = np.zeros((img_h, img_w), dtype=np.uint8)

    for class_idx, polygon in label:
        polygon = np.array(polygon)
        mask = cv2.fillPoly(mask, [polygon], color=class_idx, lineType=cv2.LINE_AA)

    save_path = os.path.join(save_root, os.path.basename(img_path).rsplit(".", 1)[0] + ".png")
    cv2.imwrite(save_path, mask)
    

生成mask.png 标签
保存目录： /media/yunfei/mm2/cicvdata/che_seg_road/seg/labels-mask/
